In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from funcs import *
from robotClass import *

import time
import numpy as np
import matplotlib.pyplot as plt
import asyncio

LOG.propagate = False

### Lab 9

In [9]:
# Setup robot
rc = setupRobot()

2022-04-13 02:47:41,932 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:B0:6C:9A:3C
2022-04-13 02:47:44,702 | INFO     |: Connected to C0:83:B0:6C:9A:3C


In [18]:
rc.pingRobot(clear = True)
rc.updatePID(20, 0.05, 0, 0, 1, 2)
#rc.move_forward([255, 255], 0, 1)

# Wait one second
await asyncio.sleep(20)

rc.stop()
rc.pingRobot(clear = False)

In [19]:
rc.imu_readings # gyroscope values

[(153.430419921875, 1649832602.174461),
 (134.62078857421875, 1649832604.589386),
 (118.93425750732422, 1649832606.914346),
 (102.1688232421875, 1649832609.2097669),
 (86.39281463623047, 1649832611.518542),
 (69.7164535522461, 1649832613.9947898),
 (52.655242919921875, 1649832616.364266),
 (37.03825759887695, 1649832618.734247),
 (21.154428482055664, 1649832621.119005)]

In [20]:
rc.tof_readings # front ToF sensor values

[(137.0, 1649832602.158852),
 (169.0, 1649832604.574222),
 (197.0, 1649832606.8994381),
 (284.0, 1649832609.2093399),
 (324.0, 1649832611.5043142),
 (219.0, 1649832613.99437),
 (193.0, 1649832616.3491251),
 (249.0, 1649832618.718993),
 (115.0, 1649832621.1040409)]

In [ ]:
rc.turn(130, 85, 0)
await asyncio.sleep(5)

# Stop
rc.stop()

In [ ]:
# Start taking measurements
rc.updatePID(20, 0.25, 0.000001, 0.75, 0, 0)
rc.pingRobot(toggleUpdate = 1, clear = True)

In [ ]:
len(rc.tof2_readings)

In [ ]:
rc.imu_readings[-25:]

In [ ]:
# Stop taking measurements
rc.pingRobot(toggleUpdate = 1)

In [ ]:
# Get front ToF sensor output
frontTOFData = rc.tof2_readings
startTimeTOF = frontTOFData[0][1]

# Get motor PWM input
motorData = rc.motor_readings

In [ ]:
# Plot ---

# Front ToF Sensor Output
plotTOFData = dataForPlot(frontTOFData)

plt.plot([x - startTimeTOF for x in plotTOFData[1]], plotTOFData[0], 'ro')

plt.title('Front ToF Sensor')
plt.xlim([0, 20])
plt.xlabel('Time (sec)')
plt.ylabel('Distance (mm)')
plt.show()

# Motor data
plotMotorData = dataForPlot(motorData)
startTimeMotor = plotMotorData[1][0]
plt.plot([x - startTimeMotor for x in plotMotorData[1]], plotMotorData[0])
plt.title('PWM Input Graph')
plt.xlabel('Time (sec)')
plt.ylabel('PWM Signal Input')
plt.show()